In [ ]:
# # Import standard libraries
import os
import subprocess
import re
import json
import codecs
# import requests
import time
import dill
time.sleep(5)

from urllib.request import urlopen

import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Date, MetaData, event, Table, text, LargeBinary, ForeignKey
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy.engine.url import URL

import psycopg2
from sqlalchemy.dialects.postgresql import insert



from src.helpers import *
from src.dbutils import *
from src.ORMutils import *
from src.models import *
from src.geo import *
from src.pdfutils import *

* ### Create the database engine that will be used throughout the rest of the notebook.

In [ ]:
# This script creates a PostgreSQL database and sets up the connection using SQLAlchemy.
# Settings
db_name = "nycdata"
user = os.getlogin()  # assumes local login user is the DB user
host = "localhost"

# Step 1: Create the database (if not exists)
def create_database_if_not_exists(db_name, user, host):
    conn = psycopg2.connect(dbname="postgres", user=user, host=host)
    conn.autocommit = True  # required for CREATE DATABASE
    cursor = conn.cursor()

    # Check if DB already exists
    cursor.execute("SELECT 1 FROM pg_database WHERE datname = %s", (db_name,))
    exists = cursor.fetchone()
    if not exists:
        cursor.execute(f"CREATE DATABASE {db_name}")
        print(f"Database '{db_name}' created.")
    else:
        print(f"Database '{db_name}' already exists.")
    try:
        cursor.execute('CREATE EXTENSION postgis;')
    except psycopg2.errors.DuplicateObject:
        pass
    cursor.close()
    conn.close()

# Step 2: Connect to the new database via SQLAlchemy
def get_engine(db_name, user, host):
    url = URL.create(
        drivername="postgresql+psycopg2",
        username=user,
        host=host,
        database=db_name,
    )
    engine = create_engine(url, echo=True)  # echo=True shows SQL queries
    return engine

# Usage
create_database_if_not_exists(db_name, user, host)
engine = get_engine(db_name, user, host)

SessionLocal = sessionmaker(bind=engine, autoflush=False, autocommit=False)

* #### Load the objects created in previous notebooks

In [ ]:
# Load the environment
with open("environment_data/table_dicts.pkl", "rb") as f:
    env = dill.load(f)

# Restore the environment
globals().update(env)

In [ ]:
dataset_info_dict

In [ ]:
metadata = MetaData()
Base.metadata.reflect(bind=engine) 

* ### Create lookup tables variables identified as categorical and for which definitions were extracted from the metadata in the previous notebook.

* There are borough codes in the PLUTO dataset, but annyoingly, in contrast to most other datasets, the borough code is a two letter inital like "BK" or "BX". Also in the PLUTO dataset, "Sanitation Borough" does use the standard numeric codes that most other NYC OpenData datasets use. All this is is to say that it requires special handling separate from my system to extract categories and create lookup tables for them programatically.

In [ ]:
lookups = {}

for name,dataset in dataset_info_dict.items():
    lookups |= {k:v for k,v in dataset.col_customizations.items() if dataset.col_customizations[k].is_category == True}

In [ ]:
lookups

In [ ]:
completed_tables = [] # This is for tracking the names of tables that have been created, which will be used to avoid creating redundant tables for columns that are same-kind repeats (such as "district_1" and "district_2"), and thus will use the same lookups.

for name,table in lookups.items():
    print(f"processing {table}")
    if table.new_name is None:
        table.new_name = table.short_name
    lookup_table_name= re.sub('_[0-9]+$', '', table.new_name)
    print(f"lookup_table_name: {lookup_table_name}")
    if any([table.new_name.startswith(prefix) and table.new_name[-1].isdigit() for prefix in completed_tables]):
        print(f"Lookup table {lookup_table_name} already created, continuing...")
        continue
    with engine.connect() as connection:
        print(f"Creating lookup table {lookup_table_name}...")
        lookup_table = create_lookup_table(Base.metadata, lookup_table_name=lookup_table_name, text_column_name='name_or_code')
        print(f"Created lookup table: {lookup_table}")
        name_prefix = lookup_table_name
        completed_tables.append(name_prefix)
        lookups[name].orm = lookup_table


Base.metadata.create_all(engine)


In [ ]:
multicolumns = {'zoning_district': 4, 'commercial_overlay': 2, 'special_purpose_district': 3}

for dataset in dataset_info_dict.values():
    for name,repetitions in multicolumns.items():
        print(name)
        print(f"Setting {name} columns")
        for k in dataset.col_customizations.keys():
            if dataset.col_customizations[k].new_name is None:
                dataset.col_customizations[k].new_name = dataset.col_customizations[k].short_name
        cols = {k:v for k,v in dataset.col_customizations.items() if dataset.col_customizations[k].new_name.startswith(name)}
        main_col = [v for k,v in dataset.col_customizations.items() if dataset.col_customizations[k].new_name.endswith("_1")]
        if main_col:
            print(f'main_col is {main_col}')
            for key in cols.keys():
                print(f'key is {key}')
                lookups[key].orm = main_col[0].orm

In [ ]:
for name,table in lookups.items():
    print(f"table.orm: {table.orm=}")
    lookup_table = table.orm
    print(f'lookup_table {lookup_table}')
    if lookup_table is None:
        print(f"Skipping {name}...")
        continue
    print(lookup_table)
    with engine.connect() as connection:
        for definition in table.definitions:
            if len(definition) == 2:
                try:
                    stmt = insert(lookup_table).values(id=int(definition[0]), name_or_code=definition[1]).on_conflict_do_nothing(index_elements=['id'])
                except ValueError:
                    stmt = insert(lookup_table).values(name_or_code=definition[0], info=definition[1]).on_conflict_do_nothing(index_elements=['id'])
            elif len(definition) == 3:
                try:
                    stmt = insert(lookup_table).values(id=int(definition[0]), name_or_code=definition[1], info=definition[2]).on_conflict_do_nothing(index_elements=['id'])
                except Exception as e:
                    print(e)
                    print(definition)
            else:
                print(definition)
                raise ValueError("Was only expecting two or three columns")
            try:
                connection.execute(stmt)
            except Exception as e:
                print(f"Error inserting into {lookup_table_name}:")
                print(e)
                print(definition)
        connection.commit()
    name_prefix = table.new_name[0:round(len(table.new_name)*.75)] # Hopefully this is a safe threshold to identify when columns are repeats of the same type

## Import the MaPLUTO data:
* List the layers in the file
* In this case there is only one layer, so it isn't necessary to know and specify which one to import, but including anyway for future reference.

In [ ]:
# Import the MapPLUTO data from geo database file (.gdb)
pluto_version = "25v1_1"
gdb_path = f"{PROJECT_DATA}/files_to_use/MapPLUTO{pluto_version}.gdb"


* Import the geodatabase (.gdb) file.

In [ ]:

geodata = {}
# List layers in the GDB file
layers = fiona.listlayers(gdb_path)
print("Layers in the GDB file:")
for layer in layers:
    print(layer)
    gdf = gpd.read_file(gdb_path, layer=layer)
    # gdf['borough'] = gdf['Borough'].replace(replacement_dict)
    try:
        gdf['wkb'] = gdf['geometry'].apply(lambda geom: geom.wkb if geom else None)
    except KeyError:
        pass
    geodata[layer] = gdf


In [ ]:
gdf = geodata[f'MapPLUTO_{pluto_version}_clipped']
is_whole_number = {(gdf[col].notna() & (gdf[col] % 1 == 0)).all() for col in gdf.columns if gdf[col].dtype == 'float'}
gdf.columns

In [ ]:
# Iterate over columns and change dtype to int where applicable
for col in gdf.columns:
    print(f'Processing column: {col} of dtype {gdf[col].dtype}')
    if gdf[col].dtype == float and is_whole_number_series(gdf[col]) and gdf[col].name not in ['wkb', 'geometry', 'Shape_Length', 'Shape_Area']:
        print(f'Column {col} is {is_whole_number_series(gdf[col])}')
        print(f'Converting {col} to integer')
        gdf[col] = gdf[col].astype('Int64')  # 'Int64' for nullable integer type in Pandas
    else:
        print(f"Skipping {col}")


In [ ]:
from sqlalchemy import inspect
inspector = inspect(engine)
print(inspector.get_table_names())  # Ensure "basement_type_or_grade_lookup" is listed

In [ ]:
col_customization_dict = dataset_info_dict['mapPLUTO'].col_customizations
rename_mappings = {v.short_name: v.new_name for v in col_customization_dict.values()}
gdf = gdf.rename(columns=rename_mappings)

for k, v in col_customization_dict.items():
    try:
        if v.new_name in gdf.columns:
            if v.dtype == 'String':
                gdf[v.new_name] = gdf[v.new_name].astype('string')
            elif v.dtype == 'Integer':
                gdf[v.new_name] = pd.to_numeric(gdf[v.new_name], errors='coerce').astype('Int64')
            elif v.dtype == 'Float':
                gdf[v.new_name] = pd.to_numeric(gdf[v.new_name], errors='coerce').astype('float64')
            if v.new_name in ['Shape_Leng', 'Shape_Area']:
                gdf['v.new_name'].astype('float64')
            # Add more dtype handling as needed
    except TypeError as e:
        print(f"TypeError for column {v.new_name}: {e}")
        print(v)


In [ ]:
# A few of the column names did not exactly match up due to slightly different field names than specified in the data dictionary, so these need to be renamed manually:
more_mappings = {
    "HealthCenterDistrict": "health_center_district",
    "SanitDistrict": "sanitation_district_number",
    "Sanitboro": "sanitation_district_boro",
    "FIRM07_FLAG": "2007_flood_insurance_rate_map_indicator",
    "PFIRM15_FLAG": "2015_preliminary_flood_insurance_rate_map",
}
gdf = gdf.rename(columns=more_mappings)
print("gdf columns:", gdf.columns.tolist())

In [ ]:
from sqlalchemy import Table, MetaData, Column, Integer, String, ForeignKey, LargeBinary, Float, Date
from sqlalchemy.orm import declarative_base
from sqlalchemy import BigInteger

# Reflect the existing database tables once
metadata.reflect(bind=engine)

# Function to map custom dtype to SQLAlchemy types
def map_custom_dtype(dtype):
    if dtype == 'Integer':
        # Use BigInteger for columns known to exceed 2^31-1
        if k in ['bbl', 'bin', 'apportionment_bbl', 'borough_tax_block_and_lot', 'tax_block', 'tax_lot', 'census_block_2020', 'census_block_2010', 'census_block']:
            return String
        else:
            return Integer
    elif dtype == 'Float' or dtype == 'Double' or dtype == 'double precision':
        return Float
    elif dtype == 'String':
        return String
    elif dtype == 'Date':
        return Date
    elif dtype == 'LargeBinary':
        return LargeBinary
    else:
        raise ValueError(f"Unsupported dtype: {dtype}")


# Function to dynamically create the table class
def create_dynamic_table_class(table_name, col_customization_dict):
    attrs = {
        '__tablename__': table_name,
        'id': Column(Integer, primary_key=True, autoincrement=True),
        'geometry': Column(String),  
        'wkb': Column(LargeBinary),  # Use LargeBinary for WKB
        'Shape_Leng' : Column(Float), # Add columns not listed in the data dictionary
        'Shape_Area' : Column(Float),
        'version_number' : Column(String),
        'apportionment_bbl': Column(String),  # Add apportionment_bbl column
        'borough_tax_block_and_lot': Column(String),  # Add borough_tax_block_and_lot column
        'community_district': Column(String), 
        'census_tract_2020': Column(String),
        'basement_type_or_grade': Column(String),
        'sanitation_district_boro': Column(String), 
        'sanitation_district_number': Column(String),
        'health_center_district': Column(String),
        '2007_flood_insurance_rate_map_indicator': Column(String),
        '2015_preliminary_flood_insurance_rate_map': Column(String),
        'police_precinct': Column(String),

    }
    attrs['__table_args__'] = {'extend_existing': True}
    
    for k, v in col_customization_dict.items():
        if v.new_name == "version_number":
            continue  # Already defined above
        if any([name for name in multicolumns if name in k]):
            k = re.sub('_[0-9]$', '', k)
        col_type = map_custom_dtype(v.dtype)
        if v.is_fk:
            attrs[k] = Column(Integer, ForeignKey(f'{v.new_name}_lookup.id'))
        elif v.is_category:
            print(f'Creating id column for {v.new_name}')
            attrs[v.new_name] = Column(col_type)
            attrs[f"{v.new_name}_id"] = Column(Integer, ForeignKey(f'{k}_lookup.id'))
        else:
            attrs[v.new_name] = Column(col_type)
    return type(table_name, (Base,), attrs)

# 2. Dynamically create ORM/table from col_customization_dict (as you do now)
MapPLUTO_Clipped = create_dynamic_table_class(f'MapPLUTO_{pluto_version}_clipped', col_customization_dict)
Base.metadata.create_all(engine)

orm_columns = {col: getattr(MapPLUTO_Clipped, col).property.columns[0].type for col in MapPLUTO_Clipped.__table__.columns.keys()}
for col in gdf.columns:
    orm_type = orm_columns.get(col)
    pd_type = gdf[col].dtype
    print(f"{col}: DataFrame dtype={pd_type}, ORM type={orm_type}")
    # Add checks for integer overflow, type mismatches, etc.
    if pd_type == 'int64' and orm_type == Integer:
        # Check for potential integer overflow
        if gdf[col].max() > 2**31 - 1:
            print(f"Warning: Potential overflow for column {col}")


In [ ]:
gdf['Shape_Leng'] = gdf['Shape_Leng'].astype('Float64')
gdf['Shape_Leng'].dtype
gdf['health_center_district'] = gdf['health_center_district'].astype('Int64')
gdf['sanitation_district_number'] = gdf['sanitation_district_number'].astype('Int64')
gdf['sanitation_district_boro'] = gdf['sanitation_district_boro'].astype('Int64')

In [ ]:


print("Column compatibility check:")
for col in gdf.columns:
    orm_type = orm_columns.get(col)
    pd_type = gdf[col].dtype
    print(f"\nColumn: {col}")
    print(f"  DataFrame dtype: {pd_type}")
    print(f"  ORM type: {orm_type}")

    # Check for float compatibility
    if "Float" in str(type(orm_type)):
        # Only check with np.issubdtype if pd_type is a numpy dtype
        if isinstance(pd_type, np.dtype):
            if not np.issubdtype(pd_type, np.floating):
                print("  WARNING: DataFrame column is not float but ORM expects Float.  Column is actually {pd_type}")
        else:
            # For extension dtypes, print a warning or skip
            print(f"  WARNING: DataFrame column is not a numpy float dtype but ORM expects Float. Column is actually {pd_type}")

    # Check for string compatibility
    if "String" in str(type(orm_type)):
        if pd_type not in ["object", "string"]:
            print(f"  WARNING: DataFrame column is not string but ORM expects String. Column is actually {pd_type}")

    # Check for LargeBinary compatibility
    if "LargeBinary" in str(type(orm_type)):
        if pd_type != "object":
            print(f"  WARNING: DataFrame column is not object but ORM expects LargeBinary. Column is actually {pd_type}")

print("\nCheck complete.")

In [ ]:
from sqlalchemy.orm import sessionmaker
from shapely import wkb

# Create a session
session = SessionLocal()

# gdf = geodata['MapPLUTO_24v4_clipped']
# def format_float(value):
#     return str(value).rstrip('0').rstrip('.') if '.' in str(value) else str(value)

import traceback

batch_size = 10000
# gdf = gdf.where(pd.notnull(gdf), None)
gdf = gdf.applymap(lambda x: None if pd.isna(x) else x)
with SessionLocal() as session:
    for start in range(0, len(gdf), batch_size):
        batch = gdf.iloc[start:start + batch_size]
        batch = batch.where(pd.notnull(batch), None)
        print("gdf columns:", gdf.columns.tolist())
        for idx, row in batch.iterrows():
            try:
                if row['apportionment_date']:
                    row['apportionment_date'] = parseDateString(row['apportionment_date'])
                for col in gdf.columns:
                    val = row[col]
                geometry_wkb = row['geometry'].wkb if row['geometry'] else None
                for col in gdf.columns:
                    if col in orm_columns and str(orm_columns[col]) == "INTEGER":
                        val = row[col]
                        if pd.isna(val):
                            row[col] = None
                        elif isinstance(val, float) and val.is_integer():
                            row[col] = int(val)
                        elif isinstance(val, (int, np.integer)):
                            row[col] = int(val)
                        else:
                            print(f"WARNING: {col} has non-integer value: {val} (type: {type(val)})")
                pluto_entry = MapPLUTO_Clipped(
                    geometry=geometry_wkb,
                    **{col: row[col] for col in gdf.columns if col not in ['geometry']}
                )
                # for key, col in pluto_entry.__table__.columns.items():
                #     val = getattr(pluto_entry, key)
                #     # Print all values
                #     print(f"{key}: {val} (type: {type(val)})")
                #     # If column is Integer in the schema, check type and value
                #     if str(col.type) == "INTEGER":
                #         if val is None:
                #             continue
                #         if isinstance(val, float):
                #             if not val.is_integer():
                #                 print(f"  ERROR: {key} is float but not integer-valued: {val}")
                #             elif val > 2_147_483_647 or val < -2_147_483_648:
                #                 print(f"  ERROR: {key} float value out of range: {val}")
                #         elif isinstance(val, (int, np.integer)):
                #             if val > 2_147_483_647 or val < -2_147_483_648:
                #                 print(f"  ERROR: {key} int value out of range: {val}")
                #         else:
                #             print(f"  ERROR: {key} has unexpected type for INTEGER column: {type(val)} value={val}")
                session.add(pluto_entry)
            except Exception as e:
                print(f"\nError at row index {idx}")
                print(f"Exception: {e}")
                print("Row data (full values):")
                for col in gdf.columns:
                    try:
                        val = row[col]
                        print(f"  {col}: {repr(val)} (type: {type(val)})")
                    except Exception as sub_e:
                        print(f"  Error printing column {col}: {sub_e}")
                print("Full traceback:")
                traceback.print_exc()
                raise e  # re-raise after logging for further debugging
        # NEW: Wrap commit in try/except to catch batch errors
        # Before session.commit()
        int_cols = [col for col in batch.columns if str(batch[col].dtype) in ['int64', 'Int64']]
        for col in int_cols:
            col_max = batch[col].max()
            col_min = batch[col].min()
            print(f"[Batch check] {col}: max={col_max}, min={col_min}, dtype={batch[col].dtype}")
            # Optionally, check for out-of-range values
            if (col_max is not np.nan and col_max > 2_147_483_647) or (col_min is not np.nan and col_min < -2_147_483_648):
                print(f"  WARNING: {col} has out-of-range value for PostgreSQL INTEGER!")
        try:
            session.commit()
        except Exception as e:
            print("\nException during session.commit()")
            print(f"Exception: {e}")
            print("Batch start index:", start)
            print("Batch end index:", start + batch_size)
            print("Dumping example rows in this batch with their version_number and types:")
            for idx, row in batch.iterrows():
                orm_type = orm_columns.get(col)
                val = row[col]
                if "Integer" in str(type(orm_type)):
                    print(f"  {col}: {val} (type: {type(val)})")
                print(f"Row {idx}:")
                for col in gdf.columns:
                    try:
                        val = row[col]
                        print(f"  {col}: {repr(val)} (type: {type(val)})")
                    except Exception as sub_e:
                        pass
                        print(f"  Error printing column {col}: {sub_e}")
            print("Full traceback:")
            traceback.print_exc()
            session.rollback()  # Rollback the session to avoid partial commits
            raise e  # re-raise after logging for further debugging
